In [1]:
import torch
import numpy as np
import sys

sys.path.append('../utils/human_body_prior/tools/')
from omni_tools import copy2cpu as c2c
from os import path as osp

both57m_dir = '../data/both57m/'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

cuda


In [2]:
sys.path.append('../utils/human_body_prior/body_model/')
from body_model import BodyModel

gender = "male"

bm_fname = '../models/smplh_models/{}/model.npz'.format(gender)
bm = BodyModel(bm_fname=bm_fname).to(comp_device)
faces = c2c(bm.f)

In [3]:
data_number = 494

both57m_npz = osp.join(both57m_dir, str(data_number), 'data.npz')
bdata = np.load(both57m_npz)
len = bdata['seq_len']

body_pose = np.zeros(63)
body_pose[45] = -np.pi/2 # left shoulder
body_pose[48] = -np.pi/2 # right shoulder
body_pose[51] = -np.pi/2 # left elbow
body_pose[52] = -np.pi/2 # left_elbow
body_pose[53] = -np.pi/2 # left_elbow
body_pose[54] = -np.pi/2 # right elbow
body_pose[55] = np.pi/2 # right elbow
body_pose[56] = np.pi/2 # right elbow
body_pose[57] = np.pi*2/3 # left wrist
body_pose[60] = np.pi*2/3 # right wrist
body_pose = np.tile(body_pose, (len, 1))

motion_parms = {
    'root_orient': torch.zeros(len,3).to(comp_device),
    'pose_body': torch.Tensor(body_pose).to(comp_device),
    'pose_hand': torch.Tensor(bdata['hand_pose'].reshape(len,-1)).to(comp_device),
    'trans': torch.zeros(len,3).to(comp_device)
}

In [4]:
import trimesh
sys.path.append('../utils/body_visualizer')
from tools.vis_tools import colors
from mesh.mesh_viewer import MeshViewer
from mesh.sphere import points_to_spheres
from tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

camera_pose = np.eye(4)
camera_pose[:3, 3] = np.array([0, 0.3, 1.5])
mv.update_camera_pose(camera_pose)

In [5]:
body_pose = bm(**{k:v for k,v in motion_parms.items() if k in ['pose_body', 'pose_hand', 'root_orient']})

def vis_body_hand_pose(fId = 0, SHOW=True):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    if SHOW: show_image(body_image)

    return body_image

body_images = []
for i in range(len):
    body_images.append(vis_body_hand_pose(fId=i, SHOW=False))

In [6]:
import mediapy as media

media.show_video(images=body_images,fps=int(bdata['framerate']),width=700)

This browser does not support the video tag.
